In [97]:
from binance.client import Client
from binance.enums import *
import os
from datetime import datetime
import requests
import pprint
import os
from dotenv import load_dotenv

#For trading
import time
import hmac
import hashlib
from urllib.parse import urlencode

In [45]:
# Step 0 - Credential Management
# Getting API key and secret
def get_credential():
    # CredentialFile Name
    CredentialFile = 'API key.env'

    # CredentialFile Path
    CredentialFile_path = os.path.join(os.getcwd(), CredentialFile)

    # Load Credential File
    LoginDetails = load_dotenv(dotenv_path=CredentialFile_path)
    
    # print("dotenv loaded successfully:", LoginDetails)
    
    API_key = os.getenv('key')
    API_secret = os.getenv('secret')
    API_Docs = os.getenv('Docs')    
    return API_key, API_secret, API_Docs

In [76]:
#testing parameters

api_key=get_credential()[0]
api_secret=get_credential()[1]
URL=get_credential()[2]

client = Client(api_key, api_secret)
client.FUTURES_URL = URL

symbol = 'BTCUSDT'
candleSize ='1m'
OrderbookSize=5

### Enquire of market data and details

In [50]:
#1. Server_timestampe -> timestampe on when we get the price,orderbook etc.
server_time = client.get_server_time()['serverTime']
print(f"Server Time: {datetime.fromtimestamp(server_time/1000)}")

Server Time: 2025-05-08 00:33:40.793000


In [54]:
#2. Get Current Price
price = client.futures_symbol_ticker(symbol=symbol)
print(f"\nCurrent Price of {symbol}: {price['price']}")


Current Price of BTCUSDT: 96501.40


In [68]:
#3. Get 2 Most Recent Candlesticks (Candle 0 is developed candlestick, Candle 1 is under development candlestick - CURRENT)
candles = client.futures_klines(symbol=symbol, interval=candleSize, limit=2)
print("\nRecent "+candleSize+" Candles:")
for i, c in enumerate(candles):
    t = datetime.fromtimestamp(c[0]/1000)
    print(f"Candle {i} - Time: {t}, Open: {c[1]}, High: {c[2]}, Low: {c[3]}, Close: {c[4]}, Volume: {c[5]}")


Recent 1m Candles:
Candle 0 - Time: 2025-05-08 00:40:00, Open: 96545.80, High: 96609.80, Low: 96545.80, Close: 96609.80, Volume: 16.990
Candle 1 - Time: 2025-05-08 00:41:00, Open: 96609.80, High: 96632.30, Low: 96609.80, Close: 96632.30, Volume: 2.411


In [84]:
#4. Get Order Book information -> TO-DO -> to make this an easier callable object
depth = client.futures_order_book(symbol=symbol, limit=OrderbookSize)
print("\nOrder Book ("+str(OrderbookSize)+"):")
print("Bids:")
for bid in depth['bids']:
    print(f"Price: {bid[0]}, Quantity: {bid[1]}")
print("Asks:")
for ask in depth['asks']:
    print(f"Price: {ask[0]}, Quantity: {ask[1]}")


Order Book (5):
Bids:
Price: 96639.60, Quantity: 0.031
Price: 96639.30, Quantity: 0.515
Price: 96638.10, Quantity: 0.004
Price: 96637.60, Quantity: 0.130
Price: 96637.40, Quantity: 0.012
Asks:
Price: 96640.00, Quantity: 7.067
Price: 96641.50, Quantity: 0.192
Price: 96641.70, Quantity: 0.002
Price: 96641.80, Quantity: 0.002
Price: 96642.10, Quantity: 0.773


In [143]:
#5. Open Orders Position
open_orders = client.futures_get_open_orders(symbol=symbol)
print(f"\nOpen Orders for {symbol}:")
pprint.pprint(open_orders)



Open Orders for BTCUSDT:
[{'avgPrice': '0',
  'clientOrderId': 'XYADD4H0zI4M75cVcX8e79',
  'closePosition': False,
  'cumQuote': '0.00000',
  'executedQty': '0',
  'goodTillDate': 0,
  'orderId': 4406234900,
  'origQty': '0.010',
  'origType': 'LIMIT',
  'positionSide': 'BOTH',
  'price': '10000',
  'priceMatch': 'NONE',
  'priceProtect': False,
  'reduceOnly': False,
  'selfTradePreventionMode': 'EXPIRE_MAKER',
  'side': 'BUY',
  'status': 'NEW',
  'stopPrice': '0',
  'symbol': 'BTCUSDT',
  'time': 1746637760437,
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'updateTime': 1746637760437,
  'workingType': 'CONTRACT_PRICE'},
 {'avgPrice': '0',
  'clientOrderId': 'fchiGk6ygXmNA9PJGz8pIw',
  'closePosition': False,
  'cumQuote': '0.00000',
  'executedQty': '0',
  'goodTillDate': 0,
  'orderId': 4406234437,
  'origQty': '0.010',
  'origType': 'LIMIT',
  'positionSide': 'BOTH',
  'price': '10000',
  'priceMatch': 'NONE',
  'priceProtect': False,
  'reduceOnly': False,
  'selfTradePreventionM

In [123]:
#6. Open Positions
positions = client.futures_position_information(symbol=symbol)
print(f"\nOpen Positions for {symbol}:")
for pos in positions:
    print(pos.keys())
for pos in positions:
    if float(pos['positionAmt']) != 0:
        print(f"Position: {pos['positionAmt']}, Entry: {pos['entryPrice']}, PnL: {pos['unRealizedProfit']}")


Open Positions for BTCUSDT:
dict_keys(['symbol', 'positionSide', 'positionAmt', 'entryPrice', 'breakEvenPrice', 'markPrice', 'unRealizedProfit', 'liquidationPrice', 'isolatedMargin', 'notional', 'marginAsset', 'isolatedWallet', 'initialMargin', 'maintMargin', 'positionInitialMargin', 'openOrderInitialMargin', 'adl', 'bidNotional', 'askNotional', 'updateTime'])
Position: 0.010, Entry: 96878.0, PnL: -1.06853097


In [121]:
pos['positionSide']

'BOTH'

In [223]:
#7. Cash Balance
balances = client.futures_account_balance()
print("\nFutures Wallet Balances:")
for bal in balances:
    if float(bal['balance']) > 0:
        print(f"{bal['asset']}: {bal['balance']} (Available: {bal['availableBalance']})")



Futures Wallet Balances:
USDT: 14999.61248800 (Available: 14948.90718505)


In [103]:
api_key=get_credential()[0]
api_secret=get_credential()[1]
URL=get_credential()[2]

'https://testnet.binancefuture.com/fapi'

### Trading

In [105]:
#8. Place Market Order
def send_market_order(symbol, side, quantity):
    url_path = '/v1/order'
    timestamp = int(time.time() * 1000)

    params = {
        'symbol': symbol,
        'side': side,  # "BUY" or "SELL"
        'type': 'MARKET',
        'quantity': quantity,
        'timestamp': timestamp
    }

    query_string = urlencode(params)
    signature = hmac.new(
        api_secret.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    url = URL + url_path + '?' + query_string + '&signature=' + signature

    headers = {
        'X-MBX-APIKEY': api_key
    }

    response = requests.post(url, headers=headers)
    print(response.status_code, response.json())

In [109]:
send_market_order(symbol=symbol, side='BUY', quantity=0.01)

200 {'orderId': 4406217485, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'njqAJYpVT0h4dM0smDaaTH', 'price': '0.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1746637132810}


In [149]:
#9. Place Limit Order; tif-> 'GTC' Good Til Canceled, 'IOC' Immediate or Cancel, 'FOK' Fill or Kill
def send_limit_order(symbol, side, quantity, price, tif='GTC'):
    timestamp = int(time.time() * 1000)

    params = {
        'symbol': symbol,
        'side': side,                # 'BUY' or 'SELL'
        'type': 'LIMIT',
        'quantity': quantity,
        'price': price,
        'timeInForce': tif,          # 'GTC', 'IOC', or 'FOK'
        'timestamp': timestamp
    }

    query_string = urlencode(params)
    signature = hmac.new(
        api_secret.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    url = f"{URL}/v1/order?{query_string}&signature={signature}"

    headers = {
        'X-MBX-APIKEY': api_key
    }

    response = requests.post(url, headers=headers)
    print(response.status_code, response.json())

In [141]:
send_limit_order(symbol=symbol, side='BUY', quantity=0.01, price='10000')

200 {'orderId': 4406234900, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'XYADD4H0zI4M75cVcX8e79', 'price': '10000.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1746637760437}


In [153]:
#10. Place Stop_Limit Order
def place_stop_limit_order(symbol, side, quantity, stop_price, limit_price):
    timestamp = int(time.time() * 1000)

    params = {
        'symbol': symbol,
        'side': side,
        'type': 'STOP',
        'stopPrice': stop_price,
        'price': limit_price,
        'quantity': quantity,
        'timeInForce': 'GTC',
        'timestamp': timestamp
    }

    query_string = urlencode(params)
    signature = hmac.new(api_secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = f"{URL}/v1/order?{query_string}&signature={signature}"

    headers = {'X-MBX-APIKEY': api_key}
    response = requests.post(url, headers=headers)
    print(response.status_code, response.json())

In [165]:
place_stop_limit_order(symbol=symbol, side='BUY', quantity=0.01, stop_price='150000', limit_price='150000')

200 {'orderId': 4406246788, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'GfKNlSZVLkdajnmqddhvHc', 'price': '150000.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '150000.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1746638190214}


In [219]:
#11. Cancel specific order
def cancel_order(symbol, order_id):
    timestamp = int(time.time() * 1000)

    params = {
        'symbol': symbol,
        'orderId': order_id,
        'timestamp': timestamp
    }

    query_string = urlencode(params)
    signature = hmac.new(
        api_secret.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    url = f"{URL}/v1/order?{query_string}&signature={signature}"
    headers = {'X-MBX-APIKEY': api_key}

    response = requests.delete(url, headers=headers)
    print(response.status_code, response.json())

In [169]:
open_orders = client.futures_get_open_orders(symbol=symbol)

In [209]:
cancel_order(symbol=symbol, order_id=open_orders[0].get('orderId'))

200 {'orderId': 4406234900, 'symbol': 'BTCUSDT', 'status': 'CANCELED', 'clientOrderId': 'XYADD4H0zI4M75cVcX8e79', 'price': '10000.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1746638454558}


In [221]:
#12. Cancel all open order
def cancel_all_open_orders(symbol):
    timestamp = int(time.time() * 1000)

    # Prepare query string
    params = {
        'symbol': symbol,
        'timestamp': timestamp
    }

    query_string = urlencode(params)
    signature = hmac.new(
        api_secret.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Build full URL
    url = f"{URL}/v1/allOpenOrders?{query_string}&signature={signature}"
    headers = {'X-MBX-APIKEY': api_key}

    # Send DELETE request
    response = requests.delete(url, headers=headers)

    if response.status_code == 200:
        print("All open orders cancelled successfully.")
        print(response.json())
    else:
        print("Failed to cancel orders:", response.status_code)
        print(response.json())


In [217]:
cancel_all_open_orders(symbol=symbol)

All open orders cancelled successfully.
{'code': 200, 'msg': 'The operation of cancel all open order is done.'}


In [225]:
#13. Stop-Loss market order
def place_stop_loss(symbol, side, quantity, stop_price):
    timestamp = int(time.time() * 1000)

    params = {
        'symbol': symbol,
        'side': side,  # 'SELL' to stop-loss a long, 'BUY' to stop-loss a short
        'type': 'STOP_MARKET',
        'stopPrice': stop_price,
        'quantity': quantity,
        'timestamp': timestamp,
        'reduceOnly': 'true'  # ensures it only reduces position
    }

    query_string = urlencode(params)
    signature = hmac.new(api_secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
    url = f"{URL}/v1/order?{query_string}&signature={signature}"
    headers = {'X-MBX-APIKEY': api_key}

    response = requests.post(url, headers=headers)
    print(response.status_code, response.json())

In [227]:
place_stop_loss(symbol, "SELL", 0.01, '80000')

200 {'orderId': 4406270458, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'FMKuaZaLxFSlSmbcmYhJG4', 'price': '0.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP_MARKET', 'reduceOnly': True, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '80000.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1746638981898}


In [231]:
client.futures_get_open_orders(symbol=symbol)

[{'orderId': 4406270458,
  'symbol': 'BTCUSDT',
  'status': 'NEW',
  'clientOrderId': 'FMKuaZaLxFSlSmbcmYhJG4',
  'price': '0',
  'avgPrice': '0',
  'origQty': '0.010',
  'executedQty': '0',
  'cumQuote': '0.00000',
  'timeInForce': 'GTC',
  'type': 'STOP_MARKET',
  'reduceOnly': True,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '80000',
  'workingType': 'CONTRACT_PRICE',
  'priceProtect': False,
  'origType': 'STOP_MARKET',
  'priceMatch': 'NONE',
  'selfTradePreventionMode': 'EXPIRE_MAKER',
  'goodTillDate': 0,
  'time': 1746638981898,
  'updateTime': 1746638981898}]

In [233]:
cancel_all_open_orders(symbol=symbol)

All open orders cancelled successfully.
{'code': 200, 'msg': 'The operation of cancel all open order is done.'}
